<a href="https://colab.research.google.com/github/Modinat-A/CE888/blob/main/one_vs_all/Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# logistic regression for multi-class classification using a one-vs-rest
import pandas as pd
%matplotlib inline
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
import nltk
from sklearn import kn
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import seaborn as sns
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
stopwords =nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

## Data Exploration

In [26]:
df.head(5)

,id,title,labels,fold
0,11882252,"cfeom1, classic familial form congenital fibro...",Female\tFibrosis\tGenetic Linkage\tGenetic Var...,3
1,11882251,algorithm program finding sequence specific ol...,Algorithms\tComputational Biology\tCupriavidus...,0
2,11922865,nerve growth factor selectively regulates expr...,Animals\tGene Expression Profiling\tGene Expre...,0
3,11879527,use breast conserving surgery: linking insuran...,"Age Factors\tAged\tAged, 80 and over\tBreast N...",0
4,11914147,assessment infectious diseases surveillance sy...,Armenia\tCommunicable Disease Control\tDisease...,7


Create Classifier Usung logistic regression

In [4]:
ovr = Pipeline([("vectorizer",
                 TfidfVectorizer(max_features = 25000)), 
                ("classifier", OneVsRestClassifier(LogisticRegression(), n_jobs = 4))])

In [5]:
def fit_model(train_df,y_train,test_df,y_test):
    # fit model
    ovr.fit(train_df, y_train)
    y_pred = ovr.predict(test_df)
    return (f1_score(y_test, y_pred, average="samples"))


def matrix(df):
      df_labels = df["labels"].values
      df_labels = [[l for l in label_string.split()] for label_string in df_labels]
      return MultiLabelBinarizer.transform(df_labels)

In [6]:
# creating a function to plit a portion nof dataset for training and testing 
def load_dataset(df, fold, all_titles):
    if all_titles != "True":
        print("Using a portion of the dataset")
        df = df[df["fold"].isin(range(0,10))]
    else:
        print("Using all dataset")

    labels = df["labels"].values
    labels = [[l for l in label_string.split()] for label_string in labels]
    multilabel_binarizer = MultiLabelBinarizer(sparse_output = True)
    multilabel_binarizer.fit(labels)
    
    def to_indicator_matrix(some_df):
        some_df_labels = some_df["labels"].values
        some_df_labels = [[l for l in label_string.split()] for label_string in some_df_labels]
        return multilabel_binarizer.transform(some_df_labels)
    test_df = df[df["fold"] == fold]
    X_test = test_df["title"].values
    y_test = to_indicator_matrix(test_df)

    train_df = df[df["fold"] != fold]
    X_train = train_df["title"].values
    y_train = to_indicator_matrix(train_df)
    
    return X_train, y_train, X_test, y_test

In [9]:
SINGLE_FOLD = True
ALL_TITLES = True

def evaluate(dataset):
    df = pd.read_csv(dataset)
    df= df.drop(['id'], axis=1)
    df['title'] = df['title'].str.lower()
    df['title'] = df['title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    
    scores = []
    for i in range(0, 10):
        train_df, y_train, test_df, y_test = load_dataset(df, i, all_titles = ALL_TITLES)
        ovr.fit(train_df, y_train)
        y_pred = ovr.predict(test_df)

        scores.append(f1_score(y_test, y_pred, average="samples"))

        if SINGLE_FOLD:
            break
    return np.mean(scores)

print("EconBiz average F-1 score:", evaluate('econbiz.csv'))

Using a portion of the dataset
EconBiz average F-1 score: 0.2056139914788414
